In [1]:
import os
import numpy as np
import json

from google.cloud import pubsub_v1
from typing import Optional
from google.api_core.exceptions import AlreadyExists



os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../erudite-flag-364413-428e8c5a8dbb.json"
project_id = "erudite-flag-364413"
topic_id = 'RC.MODEL'

publisher = pubsub_v1.PublisherClient()

topic_path = publisher.topic_path(project_id, topic_id)

try:
    topic = publisher.create_topic(name=topic_path)
    print(topic.name)
except AlreadyExists:
    print(topic_path)

projects/erudite-flag-364413/topics/RC.MODEL


In [13]:
data = {
    'a': {'rate': list(np.ones(1)), 'concentration': list(np.ones(1)), 'shape': (1,)}, 
    'b': {'rate': list(np.ones(1)), 'concentration': list(np.ones(1)), 'shape': (1,)} 
}

encode_data = json.dumps(data).encode('utf-8')
print(encode_data)

for _ in range(2):
    future = publisher.publish(topic_path, encode_data)
    future.result()

b'{"a": {"rate": [1.0], "concentration": [1.0], "shape": [1]}, "b": {"rate": [1.0], "concentration": [1.0], "shape": [1]}}'


In [7]:
def sub(project_id: str, subscription_id: str, timeout: Optional[float] = None) -> None:
    """Receives messages from a Pub/Sub subscription."""
    # Initialize a Subscriber client
    subscriber_client = pubsub_v1.SubscriberClient()
    # Create a fully qualified identifier in the form of
    # `projects/{project_id}/subscriptions/{subscription_id}`
    subscription_path = subscriber_client.subscription_path(project_id, subscription_id)

    try:
        subscription = subscriber_client.create_subscription(
            request={"name": subscription_path, "topic": topic_path}
        )
        print(subscription.name)
    except AlreadyExists:
        print(subscription_path)

    def callback(message: pubsub_v1.subscriber.message.Message) -> None:
        print(f"Received {message}.")
        # Acknowledge the message. Unack'ed messages will be redelivered.
        message.ack()
        print(f"Acknowledged {message.message_id}.")

    streaming_pull_future = subscriber_client.subscribe(
        subscription_path, callback=callback
    )
    print(f"Listening for messages on {subscription_path}..\n")

    try:
        # Calling result() on StreamingPullFuture keeps the main thread from
        # exiting while messages get processed in the callbacks.
        streaming_pull_future.result(timeout=timeout)
    except:  # noqa
        streaming_pull_future.cancel()  # Trigger the shutdown.
        streaming_pull_future.result()  # Block until the shutdown is complete.

    subscriber_client.close()

In [14]:
peer_id = '.01'
subscription_id = topic_id + peer_id

sub(project_id, subscription_id, 1)

projects/erudite-flag-364413/subscriptions/RC.MODEL.01
Listening for messages on projects/erudite-flag-364413/subscriptions/RC.MODEL.01..



In [15]:
peer_id = '.02'
subscription_id = topic_id + peer_id

sub(project_id, subscription_id, 1)

projects/erudite-flag-364413/subscriptions/RC.MODEL.02
Listening for messages on projects/erudite-flag-364413/subscriptions/RC.MODEL.02..

Received Message {
  data: b'{"a": {"rate": [1.0], "concentration": [1.0], "sha...'
  ordering_key: ''
  attributes: {}
}.
Acknowledged 7631635323652359.
Received Message {
  data: b'{"a": {"rate": [1.0], "concentration": [1.0], "sha...'
  ordering_key: ''
  attributes: {}
}.
Acknowledged 7631597399121878.
